# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902078


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/78 [00:00<?, ?it/s]

Rendering models:   3%|▎         | 2/78 [00:05<03:30,  2.77s/it]

Rendering models:   4%|▍         | 3/78 [00:05<02:28,  1.98s/it]

Rendering models:   5%|▌         | 4/78 [00:07<02:21,  1.92s/it]

Rendering models:   6%|▋         | 5/78 [00:07<01:44,  1.44s/it]

...

Rendering models:  88%|████████▊ | 69/78 [00:16<00:01,  7.80it/s]

Rendering models:  90%|████████▉ | 70/78 [00:16<00:01,  5.88it/s]

Rendering models:  96%|█████████▌| 75/78 [00:16<00:00,  7.53it/s]

Rendering models:  99%|█████████▊| 77/78 [00:16<00:00,  7.76it/s]

equidad1                              0.076352
kayleebug2017                         0.001191
JessieDawn                            0.162679
ElisabethB                            0.003074
not-logged-in-0b0736c8991a7cc2017a    0.000435
not-logged-in-587a4e9ad373b8ff5983    0.000454
Thedunkmasta                          0.002250
Thedunkmasta                          0.002241
Finlord63                             0.000497
sinjinza1984                          0.000334
not-logged-in-ab1612bc18cd7aab85db    0.000221
msalin03                              0.000143
msross                                0.000502
clizardo                              0.000646
mkirms                                0.000203
mkirms                                0.000427
djcameron                             0.000191
wthammas                              0.000130
denzel.ayala                          0.000352
not-logged-in-d4bf4d941a1b00de226b    0.000184
creinbol                              0.000428
cchen88      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())